In [ ]:
import pandas as pd
import torch
from torchvision import datasets, transforms
import numpy as np
import sys
import os
from torch.utils.data import Dataset, DataLoader
from torch import from_numpy, tensor


In [2]:
!git clone https://github.com/NicolasMakaroff/PES-metamodeling.git
%cd PES-metamodeling/src/
sys.path.append(os.getcwd())
%cd ../..

fatal: destination path 'PES-metamodeling' already exists and is not an empty directory.
/content/PES-metamodeling/src
/content


In [3]:
from data_gestion import open_data, norm, create_train_test_set

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
database = open_data('/content/drive/My Drive/Colab Notebooks/PRR/implicite-volatility.csv')   

In [ ]:
train_features, train_labels, test_features, test_labels = create_train_test_set(database,0.8,1.,'price')  
normed_train_features, normed_test_features = norm(train_features), norm(test_features)
train_dataset = pd.concat([normed_train_features, train_labels], axis=1)
test_dataset = pd.concat([normed_test_features, test_labels], axis=1)
         
train_data = train_dataset.to_numpy(dtype=np.float32) 
test_data = test_dataset.to_numpy(dtype=np.float32)   

In [7]:
  """from sklearn.preprocessing import PolynomialFeatures
  from sklearn.metrics import mean_squared_error
  from sklearn import linear_model
  for i in range(7,20):
    polyn = PolynomialFeatures(degree=i)
    x_ = polyn.fit_transform(normed_train_features) #x_ contient les degrés et produits croisés
    #Une fois qu'on a préparé les degrés on peut faire la régression
    clf = linear_model.LinearRegression()
    clf.fit(x_,train_labels)
    y_ = polyn.fit_transform(normed_test_features)
    y1 = clf.predict(y_)
    print('RMSE : {} and degree : {}' .format(mean_squared_error(y1,test_labels),i))"""

"from sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn import linear_model\nfor i in range(7,20):\n  polyn = PolynomialFeatures(degree=i)\n  x_ = polyn.fit_transform(normed_train_features) #x_ contient les degrés et produits croisés\n  #Une fois qu'on a préparé les degrés on peut faire la régression\n  clf = linear_model.LinearRegression()\n  clf.fit(x_,train_labels)\n  y_ = polyn.fit_transform(normed_test_features)\n  y1 = clf.predict(y_)\n  print('RMSE : {} and degree : {}' .format(mean_squared_error(y1,test_labels),i))"

In [ ]:
class VolatilityDataset(Dataset):
    """ PES dataset."""

    # Initialize your data, download, etc.
    def __init__(self,data):
        xy = data
        self.len = xy.shape[0]
        self.x_data = from_numpy(xy[:, 0:4])
        self.y_data = from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [ ]:
train_dataset = VolatilityDataset(train_data)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=512,
                          shuffle=False,
                          num_workers=2)

test_dataset = VolatilityDataset(test_data)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=512,
                          shuffle=False,
                          num_workers=2)

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 400)
        self.bn1 = nn.BatchNorm1d(400)
        self.fc2 = nn.Linear(400, 400)
        #self.bn2 = nn.BatchNorm1d(400)
        self.fc3 = nn.Linear(400, 400)
        #self.bn3 = nn.BatchNorm1d(400)
        self.fc4 = nn.Linear(400, 400)
        #self.bn4 = nn.BatchNorm1d(400)
        self.fc5 = nn.Linear(400,400)
        self.fc6 = nn.Linear(400,1)
        
        self.drop = nn.Dropout(p=0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))

        x = self.fc6(x)
        
        return x

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
        m.bias.data.fill_(0.0)


In [11]:
"""!git clone https://github.com/awslabs/adatune.git
%cd adatune
!python setup.py install"""

'!git clone https://github.com/awslabs/adatune.git\n%cd adatune\n!python setup.py install'

In [12]:
"""!pip install git+https://github.com/gbaydin/hypergradient-descent.git"""

'!pip install git+https://github.com/gbaydin/hypergradient-descent.git'

In [13]:
"""from hypergrad import SGDHD, AdamHD
from adatune.mu_adam import MuAdam
from adatune.network import *
from adatune.utils import *"""

'from hypergrad import SGDHD, AdamHD\nfrom adatune.mu_adam import MuAdam\nfrom adatune.network import *\nfrom adatune.utils import *'

In [ ]:
model = Regressor()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
#hyper_optim = MuAdam(optimizer, 1e-6, 1000.0, 'store_true', 0.99999, 1e-6, 'cuda')

In [ ]:
epochs = 1500
test_loss_min = np.Inf
steps = 0
model.apply(init_weights)
model.to('cuda')
train_losses, test_losses = [], []
#vg = ValidationGradient(test_loader, criterion, 'cuda')
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=200)
for e in range(epochs):
    running_loss = 0
    for features, labels in train_loader:
        optimizer.zero_grad() # dé-commenter si on utilise pas Adatune
        features,labels = features.to('cuda'), labels.to('cuda')
        MSE = model(features)
        loss = criterion(MSE, labels)

        # a commenter si on utilise pas adatune
        ###
        """first_grad = ag.grad(loss, model.parameters(), create_graph=True, retain_graph=True)
        hyper_optim.compute_hg(model, first_grad)
        for params, gradients in zip(model.parameters(), first_grad):
            params.grad = gradients
        optimizer.step()
        hyper_optim.hyper_step(vg.val_grad(model))
        clear_grad(model)"""
        ###
        
        # décommenter pour utiliser adamHD
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 2000)
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0

        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for features, labels in test_loader:
                features,labels = features.to('cuda'), labels.to('cuda')
                MSE = model(features)
                test_loss += criterion(MSE, labels)
        #scheduler.step(test_loss)
        model.train()        
        train_losses.append(running_loss/len(train_loader))
        test_loss = test_loss/len(test_loader)
        test_losses.append(test_loss/len(test_loader))
        scheduler.step(test_loss)
        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(np.log(running_loss/len(train_loader))),
              "Test Loss: {:.3f}.. ".format(torch.log(test_loss)),
              "Learning rate: {}..".format(optimizer.param_groups[0]['lr']))
        # save model if validation loss has decreased
        if test_loss <= test_loss_min:
            print('Validation loss decreased ({} --> {}).  Saving model ...'.format(test_loss_min,test_loss))
            torch.save(model.state_dict(), 'model5Percent.pt')
            test_loss_min = test_loss

Epoch: 1/1500..  Training Loss: -6.312..  Test Loss: -11.684..  Learning rate: 0.001..
Validation loss decreased (inf --> 8.430831258010585e-06).  Saving model ...
Epoch: 2/1500..  Training Loss: -11.427..  Test Loss: -11.980..  Learning rate: 0.001..
Validation loss decreased (8.430831258010585e-06 --> 6.266423497436335e-06).  Saving model ...
Epoch: 3/1500..  Training Loss: -11.139..  Test Loss: -12.005..  Learning rate: 0.001..
Validation loss decreased (6.266423497436335e-06 --> 6.1146774896769784e-06).  Saving model ...
Epoch: 4/1500..  Training Loss: -11.081..  Test Loss: -12.047..  Learning rate: 0.001..
Validation loss decreased (6.1146774896769784e-06 --> 5.860484634467866e-06).  Saving model ...
Epoch: 5/1500..  Training Loss: -11.363..  Test Loss: -12.392..  Learning rate: 0.001..
Validation loss decreased (5.860484634467866e-06 --> 4.1510897972329985e-06).  Saving model ...
Epoch: 6/1500..  Training Loss: -11.497..  Test Loss: -12.386..  Learning rate: 0.001..
Epoch: 7/1500

In [ ]:
plt.plot(test_losses)
plt.plot(train_losses)